In [1]:
import pandas as pd
import numpy as np
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
stop_words = stopwords.words("english")

### Cleaning data ready for preprocessing

In [3]:
data = pd.read_csv('data/FIREyFemmes_subs.csv')
text = data['selftext']
text.drop_duplicates(inplace = True)
text.dropna()
text = text[text != '[removed]']
#works

In [4]:
text.head()

0    -Made a throwaway account for this post-\n \nP...
1    Hello, happy Monday :)  How is the start of yo...
3    I mostly want to vent, but I'd love to hear fr...
4    First, I’m not sure is wealthy is even the cor...
5    Hello!      \n\nAny recent triumphs you're pro...
Name: selftext, dtype: object

In [17]:
text

0     -Made a throwaway account for this post-\n \nP...
1     Hello, happy Monday :)  How is the start of yo...
3     I mostly want to vent, but I'd love to hear fr...
4     First, I’m not sure is wealthy is even the cor...
5     Hello!      \n\nAny recent triumphs you're pro...
6     Am struggling with work exhaustion due to mult...
7     Just got a new job with a 115k base salary and...
8     A little bit of context. I'm 34F and make a de...
9     Hi all. \n\nI started mildly churning credit c...
10    We're getting through the week!      \n\nAny w...
12    I work at a super lax fully remote company wit...
13    Hello!      \n\nHow is your day going? What ar...
14    Happy Friday!      \n\nWhat sorts of things ar...
15    Hey! I’m conducting research on the impact of ...
16     Hope your weekend is going well!      \n\nAny...
18    Hello! I moved to a new city about a year ago ...
20    TL;DR: I need help deciding on keeping an inhe...
21    Hello!      \n\nWhat are your goals for th

In [25]:
# remove punctuation 
# remove numbers
# remove things like /n


# work out if it will work as rows in df or if i need to combine it into one big thing

In [89]:
def initial_clean(text):
    commonwords = ['Hey', 'Hi', 'Hello', 'hey', 'hi', 'hello', 'I', 'So', 'Im', 'Ive']   
    text = text.str.replace('[^\w\s]','')
    text = text.str.replace('[~^0-9]','')
    text = text.str.replace('\n','')
    text = text.str.replace("|".join(commonwords),'')
    
    return text
 
#doesnt work lmao 

In [28]:
text2 = text.str.replace('[^\w\s]','') #need to redo so that it  doesn't remove $ 

In [29]:
text2

0     Made a throwaway account for this post\n \nPol...
1     Hello happy Monday   How is the start of your ...
3     I mostly want to vent but Id love to hear from...
4     First Im not sure is wealthy is even the corre...
5     Hello      \n\nAny recent triumphs youre proud...
6     Am struggling with work exhaustion due to mult...
7     Just got a new job with a 115k base salary and...
8     A little bit of context Im 34F and make a dece...
9     Hi all \n\nI started mildly churning credit ca...
10    Were getting through the week      \n\nAny wor...
12    I work at a super lax fully remote company wit...
13    Hello      \n\nHow is your day going What are ...
14    Happy Friday      \n\nWhat sorts of things are...
15    Hey Im conducting research on the impact of co...
16     Hope your weekend is going well      \n\nAny ...
18    Hello I moved to a new city about a year ago a...
20    TLDR I need help deciding on keeping an inheri...
21    Hello      \n\nWhat are your goals for thi

In [30]:
text2 = text2.str.replace('[~^0-9]','')

In [31]:
text2 = text2.str.replace('\n','')


In [32]:
text2.dropna()

0     Made a throwaway account for this post Polymat...
1     Hello happy Monday   How is the start of your ...
3     I mostly want to vent but Id love to hear from...
4     First Im not sure is wealthy is even the corre...
5     Hello      Any recent triumphs youre proud of ...
6     Am struggling with work exhaustion due to mult...
7     Just got a new job with a k base salary and my...
8     A little bit of context Im F and make a decent...
9     Hi all I started mildly churning credit cards ...
10    Were getting through the week      Any workrel...
12    I work at a super lax fully remote company wit...
13    Hello      How is your day going What are you ...
14    Happy Friday      What sorts of things are you...
15    Hey Im conducting research on the impact of co...
16     Hope your weekend is going well      Any fun ...
18    Hello I moved to a new city about a year ago a...
20    TLDR I need help deciding on keeping an inheri...
21    Hello      What are your goals for this mo

In [32]:
# remove stop words

66


In [54]:
text2.head()

0    Made a throwaway account for this post Polymat...
1    Hello happy Monday   How is the start of your ...
3    I mostly want to vent but Id love to hear from...
4    First Im not sure is wealthy is even the corre...
5    Hello      Any recent triumphs youre proud of ...
Name: selftext, dtype: object

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [50]:
print(cleantext)

[]


In [11]:
def remove_stops(text, stop_words):
    text = str(text)

    #removes all stop words, including months
    words = text.split()
    final = []
    for word in words:
        if word not in stop_words:
            final.append(word)
            
    #eliminates double white spaces
    while "  " in final:
        final = final.replace("  ", " ")
    return (final)
    print(final)

In [12]:
def clean_docs(text, stop_words):
    
    #empty new list to store the cleaned docs
    cleanedtext = []
    
    #iterate over all docs and cleans them to be a
    #single sentence with no unwanted words

    clean_doc = remove_stops(text, stop_words)
    cleanedtext.append(clean_doc)
    
    return (cleanedtext)

In [64]:
test = clean_docs(text2, stop_words)

In [68]:
print(test[0])

['0', 'Made', 'throwaway', 'account', 'post', 'Polymat...', '1', 'Hello', 'happy', 'Monday', 'How', 'start', '...', '3', 'I', 'mostly', 'want', 'vent', 'Id', 'love', 'hear', 'from...', '4', 'First', 'Im', 'sure', 'wealthy', 'even', 'corre...', '5', 'Hello', 'Any', 'recent', 'triumphs', 'youre', 'proud', '...', '6', 'Am', 'struggling', 'work', 'exhaustion', 'due', 'mult...', '7', 'Just', 'got', 'new', 'job', 'k', 'base', 'salary', 'my...', '8', 'A', 'little', 'bit', 'context', 'Im', 'F', 'make', 'decent...', '9', 'Hi', 'I', 'started', 'mildly', 'churning', 'credit', 'cards', '...', '10', 'Were', 'getting', 'week', 'Any', 'workrel...', '12', 'I', 'work', 'super', 'lax', 'fully', 'remote', 'company', 'wit...', '13', 'Hello', 'How', 'day', 'going', 'What', '...', '14', 'Happy', 'Friday', 'What', 'sorts', 'things', 'you...', '15', 'Hey', 'Im', 'conducting', 'research', 'impact', 'co...', '16', 'Hope', 'weekend', 'going', 'well', 'Any', 'fun', '...', '18', 'Hello', 'I', 'moved', 'new', 'city

In [76]:
### COME BACK TO LATER TO USE LAMBDA FOR CLEANING 

# Load the regular expression library
#import re
# Remove punctuation
#text3['selftext'] = text['selftext'].apply(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
#text3['selftext'] = text3['selftext'].map({lambda x: x.lower()})
# Print out the first rows of papers
#text3.head()

In [77]:

id2word = corpora.Dictionary(test)
# Create Corpus
texts = test
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 11), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


In [78]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

#works

[(0,
  '0.006*"I" + 0.005*"..." + 0.005*"Hello" + 0.005*"Im" + 0.004*"Hi" + '
  '0.004*"job" + 0.004*"start" + 0.004*"company" + 0.004*"Ive" + 0.004*"Id"'),
 (1,
  '0.008*"I" + 0.006*"..." + 0.006*"Hello" + 0.005*"Hi" + 0.005*"Im" + '
  '0.005*"company" + 0.004*"job" + 0.004*"Ive" + 0.004*"start" + 0.004*"Any"'),
 (2,
  '0.028*"I" + 0.023*"..." + 0.016*"Hello" + 0.013*"Hi" + 0.008*"Im" + '
  '0.007*"job" + 0.007*"company" + 0.007*"start" + 0.007*"Any" + 0.006*"What"'),
 (3,
  '0.040*"I" + 0.027*"..." + 0.019*"Hello" + 0.012*"Im" + 0.012*"Hi" + '
  '0.009*"company" + 0.008*"Ive" + 0.008*"job" + 0.007*"Any" + 0.007*"start"'),
 (4,
  '0.035*"I" + 0.018*"Hello" + 0.016*"..." + 0.013*"Hi" + 0.010*"Im" + '
  '0.007*"Ive" + 0.007*"company" + 0.006*"job" + 0.006*"What" + 0.006*"start"'),
 (5,
  '0.004*"I" + 0.004*"Hello" + 0.004*"..." + 0.004*"Hi" + 0.004*"Im" + '
  '0.004*"Ive" + 0.004*"company" + 0.004*"job" + 0.004*"Any" + 0.004*"start"'),
 (6,
  '0.024*"I" + 0.017*"..." + 0.013*"Hello" + 0

In [87]:

# Visualize the topics
pyLDAvis.enable_notebook()

gensimvis.prepare(lda_model, corpus, id2word)
#works

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.014938 -0.001084       1        1  99.730476
8      0.012780  0.001772       2        1   0.029954
3      0.008118 -0.000803       3        1   0.029949
4      0.002858  0.000315       4        1   0.029947
2      0.002199 -0.000339       5        1   0.029947
6     -0.000077  0.000007       6        1   0.029946
9      0.000190  0.000217       7        1   0.029946
1     -0.010383 -0.000251       8        1   0.029945
0     -0.013238  0.000040       9        1   0.029945
5     -0.017385  0.000127      10        1   0.029945, topic_info=         Term       Freq      Total Category  logprob  loglift
64          I  15.000000  15.000000  Default  30.0000  30.0000
0         ...   9.000000   9.000000  Default  29.0000  29.0000
59      Hello   8.000000   8.000000  Default  28.0000  28.0000
61         Hi   4.000000   4.000000  Default  27.0000  27.0000
67         Im   4.000000   4.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
196    posted   0.000391   1.811967  Topic10  -5.5413  -0.3275
191  personal   0.000391   1.928749  Topic10  -5.5413  -0.3900
71       Just   0.000391   1.891259  Topic10  -5.5413  -0.3704
93     anyone   0.000391   1.977767  Topic10  -5.5413  -0.4151
160         k   0.000391   1.817532  Topic10  -5.5413  -0.3306

[630 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         1  0.986584         ...
2         1  0.944216           1
5         1  1.017155          13
8         1  0.932178          16
9         1  1.054640          18
...     ...       ...         ...
245       1  1.244417     weekend
247       1  0.778250      wit...
248       1  1.098290        work
249       1  0.987238  workrel...
250       1  1.141479       would

[124 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 9, 4, 5, 3, 7, 10, 2, 1, 6])

In [19]:
from pprint import pprint

In [97]:
commonwords = [' Hey ', ' Hi ', 'Hi ', 'Hi,' 'Hello', ' hey ', ' hi ', 'hello', 'Id', 'month', 'also', ' lot ', 'like', ' the ', 'would', 'could', 
               'really', ' get ', ' I ', 'I ', '\.\.\.', ' So ', 'Im', 'Ive', ' How ', 'Any', 'What', 'All ', 'Just', 'week', ' A ', 'going',
               ' ts '] 
trial2 = text2.str.replace("|".join(commonwords),'')

In [61]:
# test2 = trial2.apply(lambda text: remove_stops(text, stop_words))

In [98]:
# test2 = clean_docs(trial2, stop_words)
test2 = trial2.apply(lambda text: remove_stops(text, stop_words)) # NEED TO USE THIS LAMDBA
id2word2 = corpora.Dictionary(test2)
# Create Corpus
texts2 = test2
# Term Document Frequency
corpus2 = [id2word2.doc2bow(text) for text in texts2]
# View
print(corpus2[:1][0][:30])

[(0, 1), (1, 1), (2, 4), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 6), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 12), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 2), (27, 1), (28, 2), (29, 1)]


In [88]:
test2

0     [Made, throwaway, account, post, Polymath, FIR...
1     [Hello, happy, Monday, isstart, keeping, motiv...
3     [mostly, want, vent, love, hear, anyone, simil...
4     [First, sure, wealthy, evencorrect, wordwant, ...
5     [Hello, recent, triumphs, youre, proud, Feel, ...
6     [Am, struggling, work, exhaustion, due, multip...
7     [got, new, job, k, base, salary, OTE, yearis, ...
8     [A, little, bit, context, F, make, decent, liv...
9     [allstarted, mildly, churning, credit, cards, ...
10    [Were, getting, workrelated, matters, youd, to...
12    [work, super, lax, fully, remote, company, lai...
13    [Hello, day, thankful, todaygenerally, Feel, f...
14    [Happy, Friday, sorts, things, looking, forwar...
15    [Hey, conducting, research, onimpact, covid, w...
16    [Hope, end, well, fun, plans, Feel, free, disc...
18    [Hellomoved, new, city, year, ago, pay, nice, ...
20    [TLDRneed, help, deciding, keeping, inherited,...
21                    [Hello, goals, goals, last

In [99]:
# number of topics
num_topics2 = 5
# Build LDA model
lda_model2 = gensim.models.LdaMulticore(corpus=corpus2,
                                       id2word=id2word2,
                                       num_topics=num_topics2)
# Print the Keyword in the 10 topics
pprint(lda_model2.print_topics())
doc_lda2 = lda_model2[corpus2]

#works

[(0,
  '0.006*"job" + 0.006*"years" + 0.005*"My" + 0.005*"k" + 0.004*"much" + '
  '0.004*"work" + 0.004*"time" + 0.003*"year" + 0.003*"good" + 0.003*"income"'),
 (1,
  '0.006*"job" + 0.004*"My" + 0.004*"k" + 0.004*"time" + 0.004*"help" + '
  '0.004*"year" + 0.004*"work" + 0.003*"start" + 0.003*"much" + 0.003*"The"'),
 (2,
  '0.005*"year" + 0.004*"years" + 0.004*"time" + 0.004*"income" + 0.004*"free" '
  '+ 0.004*"much" + 0.003*"work" + 0.003*"job" + 0.003*"getting" + '
  '0.003*"make"'),
 (3,
  '0.006*"work" + 0.006*"k" + 0.005*"job" + 0.005*"My" + 0.004*"year" + '
  '0.003*"much" + 0.003*"time" + 0.003*"make" + 0.003*"The" + 0.003*"pay"'),
 (4,
  '0.006*"much" + 0.005*"job" + 0.005*"k" + 0.004*"years" + 0.004*"My" + '
  '0.003*"make" + 0.003*"The" + 0.003*"time" + 0.003*"year" + 0.003*"work"')]


In [40]:
doc_lda2 = lda_model2[corpus2]

In [100]:
# Visualize the topics
pyLDAvis.enable_notebook()

gensimvis.prepare(lda_model2, corpus2, id2word2)
#works

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.000494  0.000278       1        1  31.671492
4     -0.003836 -0.017068       2        1  19.457422
1     -0.007671  0.015110       3        1  16.901506
2      0.024986  0.001906       4        1  16.474880
3     -0.012985 -0.000226       5        1  15.494700, topic_info=           Term       Freq      Total Category  logprob  loglift
228        free   9.000000   9.000000  Default  30.0000  30.0000
540     matters   5.000000   5.000000  Default  29.0000  29.0000
520        work  20.000000  20.000000  Default  28.0000  28.0000
527        year  19.000000  19.000000  Default  27.0000  27.0000
541      thread   6.000000   6.000000  Default  26.0000  26.0000
...         ...        ...        ...      ...      ...      ...
358         one   1.831930  11.859531   Topic5  -6.1247  -0.0031
425   something   1.703902  10.119945   Topic5  -6.1971   0.0831
113        back   1.653492   9.387007   Topic5  -6.2271   0.1282
1086      right   1.662826   9.586121   Topic5  -6.2215   0.1129
16         FIRE   1.666053  11.954824   Topic5  -6.2196  -0.1060

[379 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2476      4  0.815950    ASAPd
2462      2  0.705524      All
2463      2  0.702307     Also
602       1  0.462874  America
602       2  0.462874  America
...     ...       ...      ...
529       4  0.174151    years
529       5  0.087075    years
1382      1  0.625886    yield
1222      4  0.816000     youd
1383      1  0.627399     zero

[623 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 2, 3, 4])

In [ ]:
### TO DO 

# go back to preprocessing - need to preprocess much more finely. this includes using stems to remove, working on dealing with
# numbers and ensuring meaning doesn't get fucked up due to removal of words. Stop words are a big issue here. Also need to 
# learn regex so that I can cope with it. 

# then want to add in bigrams and trigrams for common combinations of words - see digital humanities youtube

# look into tf-tdf or whatever it is called modelling instead 

# when i am ok with this i need to add in more data from comments/make new notebook and trial on this bigger dataset. Can probably 
# triple the dataset size to start with 